# Diagonalización


En este documento ilustraremos cómo usar `R` para resolver los problemas tipo propuestos por L. Merino y E. Santos en  [página de resolución de ejercicios tipo](https://www.ugr.es/~lmerino/ALME.html) correspondientes al bloque "Diagonalización".

## Ejemplo

*Dada la matriz*

$$
A=\left(
\begin{array}{ccc}
5 & 1 & -3 \\
1 & 5 & -3 \\
2 & 2 & -2 
\end{array}\right).
$$

*Determinar si es o no diagonalizable y, en caso de que lo sea, determinar su forma diagonal y una matriz de paso.*

In [1]:
A<-cbind(c(5,1,-3),c(1,5,-3),c(2,2,-2))
A

5,1,2
1,5,2
-3,-3,-2


En `R` podemos usar la función `eigen` para encontrar los valores propios y la matriz de paso.

In [2]:
vp<-eigen(A)
vp

eigen() decomposition
$values
[1] 4 4 0

$vectors
              [,1]       [,2]       [,3]
[1,]  1.250637e-16 -0.8451543 -0.3015113
[2,]  8.944272e-01 -0.1690309 -0.3015113
[3,] -4.472136e-01  0.5070926  0.9045340


La matriz de paso se almacena en `$vectors`.

In [3]:
P<-vp$vectors
zapsmall(P)

0.0000000,-0.8451543,-0.3015113
0.8944272,-0.1690309,-0.3015113
-0.4472136,0.5070926,0.9045340


Comprobemos que efectivamente $P^{-1}AP$ es diagonal.

In [4]:
solve(P)%*%A%*%P

4.000000e+00,-1.110223e-16,6.661338e-16
2.220446e-16,4.000000e+00,4.440892e-16
0.000000e+00,0.000000e+00,0.000000e+00


In [5]:
zapsmall(.Last.value)

4,0,0
0,4,0
0,0,0


Hagamos este proceso paso a paso usando `pracma`.

In [6]:
library(pracma)

Empezamos calculando el polinomio característico de `A` y sus raíces (valores propios).

In [7]:
p<-charpoly(A)
p

[1]  1 -8 16  0

In [8]:
roots(p)

[1] 0 4 4

Por lo que 0 tiene multiplicidad 1, y 4 multiplicidad 2.

Calculemos ahora los subespacios propios asociados a cada uno de esos valores propios.

In [9]:
V0<-null(A)
V0

-0.3015113
-0.3015113
0.9045340


In [10]:
V4<-null(A-4*diag(3))
V4

-0.4082483,-0.8164966
0.8816497,-0.2367007
-0.2367007,0.5265986


In [11]:
P<-cbind(V0,V4)
P

-0.3015113,-0.4082483,-0.8164966
-0.3015113,0.8816497,-0.2367007
0.9045340,-0.2367007,0.5265986


In [12]:
zapsmall(solve(P)%*%A%*%P)

0,0,0
0,4,0
0,0,4


O bien usando nuestra función `gensec` que ya vimos en el bloc de espacios vectoriales.

In [13]:
gensec<-function(A){
    # A es una matrix cuyas filas son los generadores del subespacio 
    # (o los coeficientes de las ecuaciones que lo definen)                                       
    c<-ncol(A) # número columnas de A                                                                           
    f<-nrow(A) # número de filas de A                                                                           
    r<-Rank(A) # rango de A                                                                                     
    rtAI<-rref(cbind(t(A),diag(c))) # trasponemos A y le añadimos la identidad por columnas                     
    ecs<-rtAI[(r+1):c,(f+1):(f+c)] # nos quedamos con la parte de ceros que corresponden
                                   # a las ecuaciones (o a los generadores dependiendo de la entrada)       
    return(ecs)
}

In [14]:
gV0<-gensec(A)
gV0

[1]  1  1 -3

In [15]:
gV4<-gensec(A-4*diag(3))
gV4

1,0,-0.5
0,1,-0.5


Como `gensec` representa los generadores por filas, tenemos que transponer para obtener la matriz de paso.

In [16]:
Q<-t(rbind(gV0,gV4))
Q

gV0,,
1,1.0,0.0
1,0.0,1.0
-3,-0.5,-0.5


In [17]:
zapsmall(solve(Q)%*%A%*%Q)

,gV0,,
gV0,0,0,0
,0,4,0
,0,0,4


### Ejercicio

*Estudiar si la siguiente matriz es o no diagonalizable y, en tal caso, determinar su forma diagonal y una matriz de paso.*
$$
A=\left(
\begin{array}{ccc}
2 & 1 & -2 \\
0 & 2 & -1 \\
0 & 0 & 1 
\end{array}\right)
$$

In [18]:
A<-rbind(c(2,1,-2),c(0,2,-1),c(0,0,1))
A

2,1,-2
0,2,-1
0,0,1


Al ser triangular, ya sabemos cuáles van a ser los valores propios (2 y 1).

In [19]:
p<-charpoly(A)
p

[1]  1 -5  8 -4

In [20]:
roots(p)

[1] 2 2 1

El valor propio 2 tiene multiplicidad algebraica 2, pero sin embargo, su subespacio propio asociado tiene dimensión uno, por lo que $A$ no es diagonalizable.

In [21]:
null(A-2*diag(3))

1.000000e+00
-2.220446e-16
0.000000e+00


Hay que tener en cuenta que `eigen` no da error en este caso. Simplemente, los vectores que devuelve no son una base.

In [22]:
vp<-eigen(A)
vp

eigen() decomposition
$values
[1] 2 2 1

$vectors
     [,1]          [,2]      [,3]
[1,]    1 -1.000000e+00 0.5773503
[2,]    0  4.440892e-16 0.5773503
[3,]    0  0.000000e+00 0.5773503


In [23]:
P<-vp$vectors
zapsmall(P)

1,-1,0.5773503
0,0,0.5773503
0,0,0.5773503


In [24]:
Rank(P)

[1] 2

### Ejercicio

Razonar que la siguiente matriz $A$ es diagonalizable y determinar su forma diagonal y una matriz de paso. Determinar $A^k$ en función de $k$.
$$
A=\left(
\begin{array}{cccc}
1 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & -2 \\
1 & 0 & -2 & 5 
\end{array}\right)
$$

In [25]:
A<-rbind(c(1,0,0,1),c(0,1,0,0),c(0,0,1,-2),c(1,0,-2,5))
A

1,0,0,1
0,1,0,0
0,0,1,-2
1,0,-2,5


In [26]:
vp<-eigen(A)
vp

eigen() decomposition
$values
[1]   6.000000e+00   1.000000e+00   1.000000e+00 -2.656934e-307

$vectors
           [,1] [,2]          [,3]       [,4]
[1,] -0.1825742    0  8.944272e-01 -0.4082483
[2,]  0.0000000   -1  0.000000e+00  0.0000000
[3,]  0.3651484    0  4.472136e-01  0.8164966
[4,] -0.9128709    0 -2.482534e-16  0.4082483


In [27]:
Rank(vp$vectors)

[1] 4

In [28]:
P<-vp$vectors

In [29]:
zapsmall(solve(P)%*%A%*%P)

6,0,0,0
0,1,0,0
0,0,1,0
0,0,0,0


Por lo que ya tenemos la matriz de paso y la diagonal. De esta forma $A=P D P^{-1}$, por lo que $A^k=P D^k P^{-1}$. Para calcular la expresión general haremos uso de `caracas`, y así tener las expresiones de la matriz de paso como racionales.

In [30]:
library("caracas")


Attaching package: ‘caracas’


The following objects are masked from ‘package:pracma’:

    inv, nullspace, pinv, rref, taylor


The following objects are masked from ‘package:base’:

    %*%, det, diag, diag<-




In [31]:
As<-as_sym(A)

In [32]:
As

[caracas]: ⎡1  0  0   1 ⎤
           ⎢            ⎥
           ⎢0  1  0   0 ⎥
           ⎢            ⎥
           ⎢0  0  1   -2⎥
           ⎢            ⎥
           ⎣1  0  -2  5 ⎦

In [45]:
eigenval(As)

[[1]]
[[1]]$eigval
[caracas]: 6

[[1]]$eigmult
[1] 1


[[2]]
[[2]]$eigval
[caracas]: 1

[[2]]$eigmult
[1] 2


[[3]]
[[3]]$eigval
[caracas]: 0

[[3]]$eigmult
[1] 1



In [33]:
Vs0<-nullspace(As)
Vs0

[[1]]
[caracas]: [-1  0  2  1]ᵀ


In [34]:
Vs1<-nullspace(As-as_sym(diag(4)))
Vs1

[[1]]
[caracas]: [0  1  0  0]ᵀ

[[2]]
[caracas]: [2  0  1  0]ᵀ


In [35]:
Vs6<-nullspace(As-as_sym(6*diag(4)))
Vs6

[[1]]
[caracas]: [1/5  0  -2/5  1]ᵀ


In [36]:
Ps<-cbind(Vs6[[1]],Vs1[[1]],Vs1[[2]],Vs0[[1]])
Ps

[caracas]: ⎡1/5   0  2  -1⎤
           ⎢              ⎥
           ⎢ 0    1  0  0 ⎥
           ⎢              ⎥
           ⎢-2/5  0  1  2 ⎥
           ⎢              ⎥
           ⎣ 1    0  0  1 ⎦

In [37]:
inv(Ps)

[caracas]: ⎡1/6   0  -1/3  5/6⎤
           ⎢                  ⎥
           ⎢ 0    1   0     0 ⎥
           ⎢                  ⎥
           ⎢2/5   0  1/5    0 ⎥
           ⎢                  ⎥
           ⎣-1/6  0  1/3   1/6⎦

In [38]:
inv(Ps)%*%As%*%Ps

[caracas]: ⎡6  0  0  0⎤
           ⎢          ⎥
           ⎢0  1  0  0⎥
           ⎢          ⎥
           ⎢0  0  1  0⎥
           ⎢          ⎥
           ⎣0  0  0  0⎦

In [39]:
Dk<-diag_(c("6**k","1","1","0"))

In [40]:
Ak<-Ps%*%Dk%*%inv(Ps)
Ak

[caracas]: ⎡ k               k     k ⎤
           ⎢6    4      2   6     6  ⎥
           ⎢── + ─  0   ─ - ──    ── ⎥
           ⎢30   5      5   15    6  ⎥
           ⎢                         ⎥
           ⎢  0     1     0       0  ⎥
           ⎢                         ⎥
           ⎢     k        k        k ⎥
           ⎢2   6      2⋅6    1  -6  ⎥
           ⎢─ - ──  0  ──── + ─  ────⎥
           ⎢5   15      15    5   3  ⎥
           ⎢                         ⎥
           ⎢   k           k        k⎥
           ⎢  6          -6      5⋅6 ⎥
           ⎢  ──    0    ────    ────⎥
           ⎣  6           3       6  ⎦

In [41]:
simplify(Ak)

[caracas]: ⎡ k               k           ⎤
           ⎢6    4      2   6      k - 1 ⎥
           ⎢── + ─  0   ─ - ──    6      ⎥
           ⎢30   5      5   15           ⎥
           ⎢                             ⎥
           ⎢  0     1     0         0    ⎥
           ⎢                             ⎥
           ⎢     k        k          k   ⎥
           ⎢2   6      2⋅6    1    -6    ⎥
           ⎢─ - ──  0  ──── + ─    ────  ⎥
           ⎢5   15      15    5     3    ⎥
           ⎢                             ⎥
           ⎢               k             ⎥
           ⎢ k - 1       -6         k - 1⎥
           ⎢6       0    ────    5⋅6     ⎥
           ⎣              3              ⎦

### Ejercicio

*Estudiar para qué valores de los parámetros $a$ y $b$ es diagonalizable la matriz*
$$
A=\left(
\begin{array}{ccc}
1 & a & 0 \\
0 & 0 & b \\
0 & b & 0 
\end{array}\right).
$$


In [47]:
A<-matrix_(c("1","a","0",
             "0","0","b",
             "0","b","0"),3,3,byrow=TRUE)
A

[caracas]: ⎡1  a  0⎤
           ⎢       ⎥
           ⎢0  0  b⎥
           ⎢       ⎥
           ⎣0  b  0⎦

In [50]:
cp<-do_la(A,"charpoly")
cp

[caracas]:    2      2    3    2
           - b ⋅λ + b  + λ  - λ

In [56]:
sympy_func(cp,"factor")

[caracas]: -(b - λ)⋅(b + λ)⋅(λ - 1)

Los valores propios son $\{1,b,-b\}$. Así si $b\not\in\{0,1,-1\}$, los tres valores propios son diferentes y la matriz es diagonalizable.

Estudiemos ahora qué ocurre para $b=0$, en cuyo caso los valores propios son $1$ y $0$ con multiplicidad dos. Nos interesa por tanto ver si el subespacio propio asociado a $0$ tiene dimensión dos.

In [58]:
Ab0<-subs(A,"b",0)
Ab0

[caracas]: ⎡1  a  0⎤
           ⎢       ⎥
           ⎢0  0  0⎥
           ⎢       ⎥
           ⎣0  0  0⎦

El rango de $A=A-0\times I$ en este caso no depende de $a$.

In [61]:
Vb0_0<-nullspace(Ab0)
Vb0_0

[[1]]
[caracas]: [-a  1  0]ᵀ

[[2]]
[caracas]: [0  0  1]ᵀ


Por lo que para $b=0$, la multiplicidad algebraica y geométrica de cada valor propio coincide, siendo así `Ab0` diagonalizable.

Veamos ahora el caso $b=1$. Los valores propios son $1$, con multiplicidad dos, y $-1$.

In [62]:
Ab1<-subs(A,"b",1)
Ab1

[caracas]: ⎡1   a    0 ⎤
           ⎢           ⎥
           ⎢0   0   1.0⎥
           ⎢           ⎥
           ⎣0  1.0   0 ⎦

In [68]:
T<-Ab1-as_sym(diag(3))
T

[caracas]: ⎡0   a    0 ⎤
           ⎢           ⎥
           ⎢0  -1   1.0⎥
           ⎢           ⎥
           ⎣0  1.0  -1 ⎦

En este caso el rango de $A-I$ depende de $a$. 
- Si $a\neq 0$, el rango es dos, y la dimension del subespacio propio asociado a 1 sería 1, por lo que la matriz no sería diagonalizable.
- Si $a=0$, entonces la matriz es diagonalizable, pues el rango de $A-I$ es 1, y por tanto su núcleo tiene dimensión 2.

Para $b=-1$, tenemos como valores propios a 1 (multiplicidad 2) y -1.

In [69]:
Abm1<-subs(A,"b",-1)
Abm1

[caracas]: ⎡1   a     0  ⎤
           ⎢             ⎥
           ⎢0   0    -1.0⎥
           ⎢             ⎥
           ⎣0  -1.0   0  ⎦

In [71]:
T<-Abm1-as_sym(diag(3))
T

[caracas]: ⎡0   a     0  ⎤
           ⎢             ⎥
           ⎢0   -1   -1.0⎥
           ⎢             ⎥
           ⎣0  -1.0   -1 ⎦

Repitiéndose la distinción que hicimos para $b=1$.